# ImpPres with LLM

You have to implement in this notebook a better ImpPres classifier using an LLM.
This classifier must be implemented using DSPy.


In [1]:
from dotenv import load_dotenv
import os
import dspy
load_dotenv("grok_key.ini")  
lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])
dspy.configure(lm=lm)

In [2]:
from typing import List, Literal, Tuple

class TransformationSignature(dspy.Signature):
    premise: str = dspy.InputField()
    hypothesis: str = dspy.InputField()
    label: Literal['entailment', 'neutral', 'contradiction'] = dspy.OutputField()

class ParadigmModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(TransformationSignature)
    
    def forward(self, transformations: List[Tuple[str, str]]) -> dspy.Prediction:
        results = []
        for premise, hypothesis in transformations:
            try:
                prediction = self.predict(premise=premise, hypothesis=hypothesis)
                results.append(prediction.label)
            except Exception as e:
                print(f"Error in prediction: {e}")
                continue
        
        return dspy.Prediction(results=results)


## Load ImpPres Dataset

In [3]:
from datasets import load_dataset

sections = ['presupposition_all_n_presupposition', 
            'presupposition_both_presupposition', 
            'presupposition_change_of_state', 
            'presupposition_cleft_existence', 
            'presupposition_cleft_uniqueness', 
            'presupposition_only_presupposition', 
            'presupposition_possessed_definites_existence', 
            'presupposition_possessed_definites_uniqueness', 
            'presupposition_question_presupposition']

dataset = {}
for section in sections:
    print(f"Loading dataset for section: {section}")
    dataset[section] = load_dataset("facebook/imppres", section)

Loading dataset for section: presupposition_all_n_presupposition
Loading dataset for section: presupposition_both_presupposition
Loading dataset for section: presupposition_change_of_state
Loading dataset for section: presupposition_cleft_existence
Loading dataset for section: presupposition_cleft_uniqueness
Loading dataset for section: presupposition_only_presupposition
Loading dataset for section: presupposition_possessed_definites_existence
Loading dataset for section: presupposition_possessed_definites_uniqueness
Loading dataset for section: presupposition_question_presupposition


In [4]:
import random
section_to_split = {
    'presupposition_all_n_presupposition': 'all_n_presupposition',
    'presupposition_both_presupposition': 'both_presupposition',
    'presupposition_change_of_state': 'change_of_state',
    'presupposition_cleft_existence': 'cleft_existence',
    'presupposition_cleft_uniqueness': 'cleft_uniqueness',
    'presupposition_only_presupposition': 'only_presupposition',
    'presupposition_possessed_definites_existence': 'possessed_definites_existence',
    'presupposition_possessed_definites_uniqueness': 'possessed_definites_uniqueness',
    'presupposition_question_presupposition': 'question_presupposition',
}


def prepare_paradigm_examples(dataset, sections):
    paradigm_examples = {}
    
    for section in sections:
        paradigm_examples[section] = {}
        split_name = section_to_split[section]
        data = dataset[section][split_name]
        
        # Group by paradigmID
        paradigm_groups = {}
        for item in data:
            paradigm_id = item['paradigmID']
            if paradigm_id not in paradigm_groups:
                paradigm_groups[paradigm_id] = []
            paradigm_groups[paradigm_id].append(item)
        
        for paradigm_id, items in paradigm_groups.items():
            if len(items) == 19:  # Only use complete paradigms
                # Shuffle to avoid position bias
                random.shuffle(items)
                transformations = [(item['premise'], item['hypothesis']) for item in items]
                true_labels = [item['gold_label'] for item in items]
                paradigm_examples[section][paradigm_id] = {
                    'transformations': transformations,
                    'true_labels': true_labels,
                    'items': items
                }
    
    return paradigm_examples


## Evaluate Metrics

Let's use the huggingface `evaluate` package to compute the performance of the baseline.


In [5]:
from evaluate import load

accuracy = load("accuracy")
precision = load("precision")
recall = load("recall")
f1 = load("f1")


In [6]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

## Your Turn

Compute the classification metrics on the baseline LLM model on each test section of the ANLI dataset for samples that have a non-empty 'reason' field.

You also must show a comparison between the DeBERTa baseline model and this LLM baseline model. The comparison metric should compute the agreement between the two models:
* On how many samples they are both correct [Correct]
* On how many samples Model1 is correct and Model2 is incorrect [Correct1]
* On how many samples Model1 is incorrect and Model2 is correct [Correct2]
* On how many samples both are incorrect [Incorrect]

In [7]:
from scipy.stats import entropy
from collections import Counter

def calculate_consistency_metric(predictions):
    if not predictions:
        return 0.0
    
    label_counts = Counter(predictions)
    total = len(predictions)
    
    probabilities = [count / total for count in label_counts.values()]
    
    ent = entropy(probabilities, base=len(label_counts)) if len(label_counts) > 1 else 0
    
    # Return consistency (1 - normalized entropy)
    return 1 - ent

def calculate_accuracy_metric(predictions, true_labels):
    if len(predictions) != len(true_labels):
        return 0.0
    
    correct = sum(1 for pred, true in zip(predictions, true_labels) if pred == true)
    return correct / len(true_labels)

def combined_metric(example, pred, alpha=0.5):
    accuracy = calculate_accuracy_metric(pred.results, example['true_labels'])
    consistency = calculate_consistency_metric(pred.results)
    
    return alpha * accuracy + (1 - alpha) * consistency


In [8]:
def create_training_examples(paradigm_examples, sections, max_examples_per_section=10):
    training_examples = []
    
    for section in sections:
        paradigms = list(paradigm_examples[section].items())
        # Limit number of examples per section for training
        selected_paradigms = paradigms[:max_examples_per_section]
        
        for paradigm_id, paradigm_data in selected_paradigms:
            example = dspy.Example(
                transformations=paradigm_data['transformations'],
                true_labels=paradigm_data['true_labels']
            ).with_inputs('transformations')
            
            training_examples.append(example)
    
    return training_examples

In [9]:
import numpy as np
from tqdm import tqdm

label_names = ["entailment", "neutral", "contradiction"]
label2id = {label: i for i, label in enumerate(label_names)}

def evaluate_section(module, paradigm_examples, section):
    paradigms = paradigm_examples[section]
    
    all_predictions = []
    all_true_labels = []
    consistency_scores = []
    transformation_results = {i: {'predictions': [], 'true_labels': []} for i in range(19)}
    
    for paradigm_id, paradigm_data in tqdm(paradigms.items()):
        pred = module(paradigm_data['transformations'])
        
        all_predictions.extend(pred.results)
        all_true_labels.extend(paradigm_data['true_labels'])
        consistency = calculate_consistency_metric(pred.results)
        print(f"Paradigm {paradigm_id} consistency: {consistency}")
        consistency_scores.append(consistency)
        
        for i, (prediction, true_label) in enumerate(zip(pred.results, paradigm_data['true_labels'])):
            transformation_results[i]['predictions'].append(label2id[prediction.strip().lower()])
            transformation_results[i]['true_labels'].append(true_label)
    
    numbered_all_predictions = [label2id[pred.strip().lower()] for pred in all_predictions]
    
    total_accuracy = accuracy.compute(predictions=numbered_all_predictions, references=all_true_labels)["accuracy"]
    total_precision = precision.compute(predictions=numbered_all_predictions, references=all_true_labels, average="macro")["precision"]
    total_recall = recall.compute(predictions=numbered_all_predictions, references=all_true_labels, average="macro")["recall"]
    total_f1 = f1.compute(predictions=numbered_all_predictions, references=all_true_labels, average="macro")["f1"]
    avg_consistency = np.mean(consistency_scores) if consistency_scores else 0.0
    
    print("Calculate metrics by transformation")
    transformation_metrics = {}
    for i in tqdm(range(19)):
        if transformation_results[i]['predictions']:
            trans_acc = accuracy.compute(
                predictions=transformation_results[i]['predictions'], 
                references=transformation_results[i]['true_labels']
            )["accuracy"]
            trans_p = precision.compute(
                predictions=transformation_results[i]['predictions'], 
                references=transformation_results[i]['true_labels'], 
                average='macro', zero_division=0
            )["precision"]
            trans_r = recall.compute(
                predictions=transformation_results[i]['predictions'], 
                references=transformation_results[i]['true_labels'], 
                average='macro', zero_division=0
            )["recall"]
            trans_f1= f1.compute(
                predictions=transformation_results[i]['predictions'], 
                references=transformation_results[i]['true_labels'], 
                average='macro'
            )["f1"]
            transformation_metrics[i] = {
                'accuracy': trans_acc,
                'precision': trans_p,
                'recall': trans_r,
                'f1': trans_f1
            }
    
    return {
        'accuracy': total_accuracy,
        'precision': total_precision,
        'recall': total_recall,
        'f1': total_f1,
        'consistency': avg_consistency,
        'transformation_metrics': transformation_metrics
    }

In [10]:
from dspy import MIPROv2
label_names = ["entailment", "neutral", "contradiction"]
label2id = {label: i for i, label in enumerate(label_names)}

def exact_match_metric(example, pred, trace=None):
    count = 0
    for i in range(len(pred.results)):
        count += 1 if label2id[pred.results[i].strip().lower()] == example['true_labels'][i] else 0
    return count == len(pred.results) 

optimizer = MIPROv2(metric=exact_match_metric)

In [11]:
import pandas as pd
from tqdm import tqdm

def run_evaluation():
    print("Preparing paradigm examples...")
    paradigm_examples = prepare_paradigm_examples(dataset, sections)
    
    print("Creating training examples...")
    training_examples = create_training_examples(paradigm_examples, sections)
    
    print("Initializing module...")
    module = ParadigmModule()
    
    print("Optimizing module...")
    compiled_module = optimizer.compile(
        module, 
        trainset=training_examples, 
        requires_permission_to_run=False
    )
    return compiled_module, paradigm_examples

def evaluate_all_sections(module, paradigm_examples):
    print("Running evaluation...")
    results = {}
    
    for section in tqdm(sections):
        if section == 'presupposition_all_n_presupposition':
            continue
        print(f"Evaluating section: {section}")
        section_results = evaluate_section(module, paradigm_examples, section)
        results[section] = section_results
        
        print(f"Section {section}:")
        print(f"  Accuracy: {section_results['accuracy']:.4f}")
        print(f"  Precision: {section_results['precision']:.4f}")
        print(f"  Recall: {section_results['recall']:.4f}")
        print(f"  F1: {section_results['f1']:.4f}")
        print(f"  Consistency: {section_results['consistency']:.4f}")
        print()
    
    return results


def create_results_table(results, sections):
    section_data = []
    for section in sections:
        section_data.append({
            'Section': section,
            'Accuracy': results[section]['accuracy'],
            'Precision': results[section]['precision'],
            'Recall': results[section]['recall'],
            'F1': results[section]['f1'],
            'Consistency': results[section]['consistency']
        })
    section_df = pd.DataFrame(section_data)
    
    # Transformation-level results (averaged across sections)
    transformation_data = []
    for i in range(19):
        trans_metrics = {'transformation': i}
        for metric in ['accuracy', 'precision', 'recall', 'f1']:
            values = []
            for section in sections:
                if i in results[section]['transformation_metrics']:
                    values.append(results[section]['transformation_metrics'][i][metric])
            trans_metrics[metric] = np.mean(values) if values else 0.0
        transformation_data.append(trans_metrics)
    transformation_df = pd.DataFrame(transformation_data)
    
    return section_df, transformation_df


In [12]:
trained_module, paradigm_examples = run_evaluation()

Preparing paradigm examples...


2025/08/03 13:35:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 72

2025/08/03 13:35:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/08/03 13:35:08 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/08/03 13:35:08 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Creating training examples...
Initializing module...
Optimizing module...
Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 33%|███▎      | 6/18 [00:00<00:00, 44.09it/s]


Bootstrapped 4 full traces after 6 examples for up to 1 rounds, amounting to 6 attempts.
Bootstrapping set 4/6


...

Bootstrapped 4 full traces after 6 examples for up to 1 rounds, amounting to 6 attempts.
Bootstrapping set 6/6


  6%|▌         | 1/18 [00:00<00:00, 53.60it/s]
2025/08/03 13:35:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/08/03 13:35:08 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.


Bootstrapped 1 full traces after 1 examples for up to 1 rounds, amounting to 1 attempts.


2025/08/03 13:35:26 INFO dspy.teleprompt.mipro_optimizer_v2: 
Proposing N=3 instructions...



KeyboardInterrupt: 

In [ ]:
results = evaluate_all_sections(trained_module, paradigm_examples)

Running evaluation...


  0%|          | 0/9 [00:00<?, ?it/s]

Evaluating section: presupposition_both_presupposition


Paradigm 0 consistency: 0.01736983605538045
Paradigm 1 consistency: 0.01736983605538034
Paradigm 2 consistency: 0.01736983605538045
Paradigm 3 consistency: 0.01736983605538034
Paradigm 4 consistency: 0.01736983605538034
Paradigm 5 consistency: 0.01736983605538045
Paradigm 6 consistency: 0.01736983605538034
Paradigm 7 consistency: 0.01736983605538034
Paradigm 8 consistency: 0.01736983605538045
Paradigm 9 consistency: 0.03826325692095478


Paradigm 10 consistency: 0.047909643385225076


...

100%|██████████| 100/100 [1:45:44<00:00, 63.44s/it]

Paradigm 99 consistency: 0.16494454884084464
1900


Calculate metrics by transformation


100%|██████████| 9/9 [2:50:59<00:00, 1140.00s/it]

Section presupposition_question_presupposition:
  Accuracy: 0.8421
  Precision: 0.8741
  Recall: 0.8216
  F1: 0.8340
  Consistency: 0.1217



In [ ]:
section_df, transformation_df = create_results_table(results, 
    ['presupposition_both_presupposition',
        'presupposition_change_of_state',
        'presupposition_cleft_existence',
        'presupposition_cleft_uniqueness',
        'presupposition_only_presupposition',
        'presupposition_possessed_definites_existence',
        'presupposition_possessed_definites_uniqueness',
        'presupposition_question_presupposition'])

print("\n" + "="*80)
print("SECTION-LEVEL RESULTS")
print("="*80)
print(section_df.to_string(index=False))

print("\n" + "="*80)
print("TRANSFORMATION-LEVEL RESULTS")
print("="*80)
print(transformation_df.to_string(index=False))

# Calculate overall metrics
overall_accuracy = section_df['Accuracy'].mean()
overall_precision = section_df['Precision'].mean()
overall_recall = section_df['Recall'].mean()
overall_f1 = section_df['F1'].mean()
overall_consistency = section_df['Consistency'].mean()

print(f"\n" + "="*80)
print("OVERALL RESULTS")
print("="*80)
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1: {overall_f1:.4f}")
print(f"Overall Consistency: {overall_consistency:.4f}")


SECTION-LEVEL RESULTS
                                      Section  Accuracy  Precision   Recall       F1  Consistency
           presupposition_both_presupposition  0.976316   0.976741 0.977250 0.976923     0.023364
               presupposition_change_of_state  0.567895   0.661949 0.496972 0.470625     0.441210
               presupposition_cleft_existence  0.688421   0.848858 0.631222 0.641477     0.333441
              presupposition_cleft_uniqueness  0.478947   0.748956 0.395028 0.311998     0.687332
           presupposition_only_presupposition  0.699474   0.829189 0.655722 0.678558     0.254056
 presupposition_possessed_definites_existence  0.937368   0.944866 0.928861 0.935482     0.054856
presupposition_possessed_definites_uniqueness  0.480526   0.737844 0.396833 0.314979     0.677115
       presupposition_question_presupposition  0.842105   0.874063 0.821556 0.833981     0.121652

TRANSFORMATION-LEVEL RESULTS
 transformation  accuracy  precision   recall       f1
          

## Results Analysis

This experiment evaluates whether explanation-based prompting can improve presupposition identification by exploiting the structured paradigms encoded in the ImpPres dataset. 

Results show that while certain presupposition types (e.g., possessed definites (existence) and both presupposition) achieve high accuracy, they often lack consistency across transformations, indicating limited generalization from the paradigm structure. In contrast, other types such as cleft uniqueness and possessed definites (uniqueness) show high consistency but low accuracy, suggesting some problem in the model systematicly happening across the section, possibly caused by a specific pattern of the it, that isn't changed throught transformations. 

Transformation-level scores are relatively uniform, implying the model captures some general patterns across variants. However, the overall consistency score (0.32) remains low, limiting the effectiveness of paradigm-aware reasoning. 
The overall accuracy is 0.7 which is less then the previous task, meaning the added pradigm calculation is hurting the preformence of the model.  

These results suggest that while the model benefits from the structure of paradigms to some extent, further improvements are needed, particularly through training or prompting strategies that explicitly reinforce consistency across transformations.

